In [ ]:
!pip install torch torchvision torchaudio --upgrade
!pip install torchmetrics tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 13.8 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torchmetrics import Accuracy
from tqdm import tqdm
import os

In [ ]:
data_dir = "./data"
batch_size = 32
num_classes = 101

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_dataset = datasets.Food101(root=data_dir, split='train',
                                 download=True, transform=train_transform)
test_dataset = datasets.Food101(root=data_dir, split='test',
                                download=True, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size,
                          shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size,
                         shuffle=False, num_workers=2, pin_memory=True)

100%|██████████| 5.00G/5.00G [02:44<00:00, 30.3MB/s]


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.efficientnet_b0(weights="IMAGENET1K_V1")
in_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_features, num_classes)
model = model.to(device)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:02<00:00, 9.09MB/s]


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

In [ ]:
def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    acc = Accuracy(task="multiclass", num_classes=num_classes).to(device)

    for images, labels in tqdm(loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        acc.update(outputs, labels)

    return running_loss / len(loader.dataset), acc.compute().item()


def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    acc = Accuracy(task="multiclass", num_classes=num_classes).to(device)

    with torch.no_grad():
        for images, labels in tqdm(loader, desc="Validation", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            acc.update(outputs, labels)

    return running_loss / len(loader.dataset), acc.compute().item()

In [ ]:
num_epochs = 10
best_acc = 0.0

for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = evaluate(model, test_loader, criterion, device)
    scheduler.step()

    print(f"Epoch [{epoch+1}/{num_epochs}]")
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc*100:.2f}%")
    print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc*100:.2f}%")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_food101_efficientnetb0.pth")
        print("✅ Best model saved.")

print(f"\nTraining complete. Best validation accuracy: {best_acc*100:.2f}%")

Epoch [1/10]
Train Loss: 2.3910 | Train Acc: 44.97%
Val Loss: 0.9938 | Val Acc: 73.58%
✅ Best model saved.


Epoch [2/10]
Train Loss: 1.4848 | Train Acc: 62.36%
Val Loss: 0.7877 | Val Acc: 78.46%
✅ Best model saved.


Epoch [3/10]
Train Loss: 1.2847 | Train Acc: 67.06%
Val Loss: 0.7044 | Val Acc: 80.27%
✅ Best model saved.


Epoch [4/10]
Train Loss: 1.1489 | Train Acc: 70.27%
Val Loss: 0.6379 | Val Acc: 82.11%
✅ Best model saved.


Epoch [5/10]
Train Loss: 1.0649 | Train Acc: 72.26%
Val Loss: 0.5982 | Val Acc: 82.94%
✅ Best model saved.


Epoch [6/10]
Train Loss: 0.9986 | Train Acc: 73.91%
Val Loss: 0.5726 | Val Acc: 83.84%
✅ Best model saved.


Epoch [7/10]
Train Loss: 0.9443 | Train Acc: 75.16%
Val Loss: 0.5595 | Val Acc: 84.19%
✅ Best model saved.


Epoch [8/10]
Train Loss: 0.9055 | Train Acc: 76.23%
Val Loss: 0.5466 | Val Acc: 84.44%
✅ Best model saved.


Epoch [9/10]
Train Loss: 0.8794 | Train Acc: 76.88%
Val Loss: 0.5357 | Val Acc: 84.86%
✅ Best model saved.


Epoch [10/10]
Train Loss: 0.8698 | Train Acc: 77.07%
Val Loss: 0.5313 | Val Acc: 84.91%
✅ Best model saved.

Training complete. Best validation accuracy: 84.91%
